In [1]:
from google.colab import drive
drive.mount('/content/drive')

import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.utils import shuffle

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/train.csv')
test = pd.read_csv('/content/drive/My Drive/Colab Notebooks/test.csv')
train.head()

1. Feature engineering

먼저, cabin은 대부분 비어있어서 빼고, ticket number는 큰 의미가 없을 것 같아서 일단 빼두었습니다.


In [0]:
train= train.drop(['Cabin'],axis=1)
train= train.drop(['Ticket'],axis=1)
test= test.drop(['Cabin'],axis=1)
test= test.drop(['Ticket'],axis=1)

Embarked 정보를 보니 S, C, Q로 나뉘어 있고 2개의 NaN값이 있었는데, 각 항구별 수를 보니 S항구에서 승선한 사람이 가장 많아서 S로 채웠습니다. 그리고 Char를 숫자로 바꿔주었습니다.

In [4]:
southampton= train[train["Embarked"]=="S"].shape[0]
print("S: ",southampton)
cherbourg= train[train["Embarked"]=="C"].shape[0]
print("C: ",cherbourg)
queenstown= train[train["Embarked"]=="Q"].shape[0]
print("Q: ",queenstown)

train= train.fillna({"Embarked": "S"})
train.isnull().sum()

embarked_mapping = {"S": 1, "C": 2, "Q": 3}
train['Embarked']= train['Embarked'].map(embarked_mapping)
test['Embarked']= test['Embarked'].map(embarked_mapping)

S:  644
C:  168
Q:  77


그 다음은 이름을 몇개의 class로 분류해보았습니다.
어떤분이 통계를 낸 것을 보니, 남자보단 여자가, 다른 class보다 royal층이 생존한 확률이 높더라고요.. 
그래서 Name들을 6개의 class(Royal, Miss, Mrs, Mr, Master, Rare)로 분류하였습니다.
마찬가지로 분류한 class들을 숫자로 바꿔주었습니다.

In [0]:
train_test_data = [train, test]
for dataset in train_test_data:
    dataset['Title'] = dataset['Name'].str.extract(' ([A-Za-z]+)\.', expand = False)
train['Title'].value_counts()

In [0]:
for dataset in train_test_data:
  dataset['Title']= dataset['Title'].replace(['Lady','Capt','Col','Don','Dr','Major','Rev','Jonkheer','Dona'],'Rare')
  
  dataset['Title']= dataset['Title'].replace(['Countess','Lady','Sir'],'Royal')
  dataset['Title']= dataset['Title'].replace('Mlle','Miss')
  dataset['Title']= dataset['Title'].replace('Ms','Miss')
  dataset['Title']= dataset['Title'].replace('Mme','Mrs')
  
train['Title'].value_counts()

In [0]:
title_mapping= {"Mr":1, "Miss":2, "Mrs": 3, "Master": 4, "Royal": 5, "Rare": 6,}
for dataset in train_test_data:
  dataset['Title']= dataset['Title'].map(title_mapping)
  dataset['Title']= dataset['Title'].fillna(0)

train= train.drop(['Name', 'PassengerId'],axis=1)
passengerid= test[["PassengerId"]].values
test= test.drop(['Name', 'PassengerId'],axis=1)
combine= [train, test]
sex_mapping= {"male":0, "female": 1}
for dataset in combine :
  dataset['Sex']=dataset['Sex'].map(sex_mapping)
train.head()


그 다음 Age는 비어있는 NaN값을 각 title별로 median값을 구해 넣어주었습니다.

In [9]:
train["Age"].fillna(train.groupby("Title")["Age"].transform("median"), inplace=True)
test["Age"].fillna(test.groupby("Title")["Age"].transform("median"), inplace=True)
train.head(20)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title
0,0,3,0,22.0,1,0,7.2500,1,1
1,1,1,1,38.0,1,0,71.2833,2,3
2,1,3,1,26.0,0,0,7.9250,1,2
3,1,1,1,35.0,1,0,53.1000,1,3
4,0,3,0,35.0,0,0,8.0500,1,1
5,0,3,0,30.0,0,0,8.4583,3,1
6,0,1,0,54.0,0,0,51.8625,1,1
7,0,3,0,2.0,3,1,21.0750,1,4
8,1,3,1,27.0,0,2,11.1333,1,3
9,1,2,1,14.0,1,0,30.0708,2,3


In [0]:
train.isnull().sum()

In [0]:
train.info()

Modeling!!

In [0]:
### Modeling!
train_data = train.drop(['Survived'],axis=1)
target= train['Survived']

M=8 # # of features
K=1 # # of classes 
#Hypothesis (modeling)
x=tf.placeholder(tf.float32, shape=[None,M])
y_target=tf.placeholder(tf.float32, shape=[None,K])

#layer1
n1= 12 # dimension
w1=tf.Variable(tf.zeros([M,n1]), name='weight1')
b1=tf.Variable(tf.zeros([n1]), name='bias1')
layer1=tf.nn.sigmoid(tf.matmul(x,w1)+b1)
#layer1 = tf.nn.dropout(layer1,keep_prob=0.5) # dropout

#layer2
w2=tf.Variable(tf.zeros([n1,K]), name='weight2')
b2=tf.Variable(tf.zeros([K]), name='bias2')
y=tf.nn.sigmoid(tf.matmul(layer1,w2)+b2)

#cost/ loss function
cost= -tf.reduce_sum(y_target*tf.log(y+1e-8)+(1-y_target)*tf.log(1-y+1e-8),axis=1)
#Minimize
optimizer= tf.train.AdamOptimizer(learning_rate=0.01)
train_model= optimizer.minimize(cost) 

# Test model
is_correct = tf.equal(tf.to_float(tf.greater(y, 0.5)), y_target)
# Calculate accuracy
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
test_result = tf.to_float(tf.greater(y, 0.5))

In [11]:
"모델 학습"
#Initializes global vaiables 
sess=tf.Session()
sess.run(tf.global_variables_initializer())

#Training
batch_size= 50
for epoch in range(2000):
    X_, Y_ =shuffle(train_data,target)
    X_=np.asarray(X_).astype('float32')
    Y_=np.asarray(Y_).astype('float32')
    Y_=np.expand_dims(Y_,axis=1)
    total_batch= int(891/batch_size)
    avg_cost=0
    
    for i in range(total_batch):
        batch_xs, batch_ys = X_[i*batch_size:i*batch_size+batch_size], Y_[i*batch_size:i*batch_size+batch_size]
        c,_= sess.run([cost, train_model], feed_dict = {x : batch_xs, y_target : batch_ys})
        avg_cost+= c/ total_batch
    
    if (epoch % 100) == 0: 
        train_accuracy = sess.run(accuracy,feed_dict = {x : X_, y_target : Y_})
        print('Epoch:', '%04d' % (epoch + 1), 'acc =',  train_accuracy)
                
target=np.expand_dims(target,axis=1)        
train_accuracy = sess.run(accuracy,feed_dict = {x : train_data, y_target : target})
print("\nTrain Acc: ", train_accuracy)

Epoch: 0001 acc = 0.61616164
Epoch: 0101 acc = 0.8148148
Epoch: 0201 acc = 0.82828283
Epoch: 0301 acc = 0.8294052
Epoch: 0401 acc = 0.8305275
Epoch: 0501 acc = 0.8271605
Epoch: 0601 acc = 0.8271605
Epoch: 0701 acc = 0.8305275
Epoch: 0801 acc = 0.8215488
Epoch: 0901 acc = 0.82828283
Epoch: 1001 acc = 0.8294052
Epoch: 1101 acc = 0.8260382
Epoch: 1201 acc = 0.8305275
Epoch: 1301 acc = 0.8294052
Epoch: 1401 acc = 0.8260382
Epoch: 1501 acc = 0.82828283
Epoch: 1601 acc = 0.82828283
Epoch: 1701 acc = 0.8271605
Epoch: 1801 acc = 0.82828283
Epoch: 1901 acc = 0.8271605

Train Acc:  0.82828283


In [0]:
test.isnull().sum()

Test data에 fare에서 NaN값이 하나 있는데 Pclass와 Fare가 어느정도 연관성이 있는것 같아 Fare 데이터 빠진 값의 Pclass를 가진 사람들의 평균 Fare를 넣어주어 채웠습니다.

In [12]:
print (train[['Pclass', 'Fare']].groupby(['Pclass'], as_index=False).mean())
print("")
print(test[test["Fare"].isnull()]["Pclass"])

   Pclass       Fare
0       1  84.154687
1       2  20.662183
2       3  13.675550

152    3
Name: Pclass, dtype: int64


In [0]:
for dataset in combine:
  dataset['Fare'] = dataset['Fare'].fillna(13.675) # The only one empty fare data's pclass is 3.
test_data = test

Test!

In [14]:
test_accuracy = sess.run(test_result, feed_dict={x : test_data})
sub=np.concatenate((passengerid.astype(int), test_accuracy.astype(int)),axis=1)
sub=pd.DataFrame(sub,columns=['PassengerId','Survived'])
print(sub.head())
#sub.to_csv("submission_2_1.csv",index=False)

   PassengerId  Survived
0          892         0
1          893         0
2          894         0
3          895         0
4          896         1
